In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

def train_and_recommend(user_id, num_recommendations=5):
    # Generate random data for 10 users and 10 projects
    np.random.seed(42)
    user_ids = np.arange(1, 11)
    project_ids = np.arange(1, 11)
    data = []

    for user in user_ids:
        for project in project_ids:
            # Randomly assign investment amounts or leave as 0 to simulate missing data
            investment = np.random.choice([0, np.random.randint(1, 100)], p=[0.7, 0.3])  # 70% chance of 0
            data.append([user, project, investment])

    df = pd.DataFrame(data, columns=['user_id', 'project_id', 'investment_amount'])

    # Create the investment matrix
    investment_matrix = df.pivot(index='user_id', columns='project_id', values='investment_amount').fillna(0)
    investment_matrix_np = investment_matrix.to_numpy()

    # Perform Singular Value Decomposition (SVD)
    k = min(investment_matrix_np.shape) - 1  # Adjust k based on matrix dimensions
    U, sigma, Vt = svds(investment_matrix_np, k=k)
    sigma = np.diag(sigma)

    # Predict missing investment amounts
    predicted_investments = np.dot(np.dot(U, sigma), Vt)
    predicted_df = pd.DataFrame(predicted_investments, columns=investment_matrix.columns, index=investment_matrix.index)

    # Function to recommend projects based on predicted investments
    def recommend_projects(user_id, num_recommendations=5):
        if user_id not in predicted_df.index:
            print(f"User ID {user_id} not found in data.")
            return {}
        user_predictions = predicted_df.loc[user_id].sort_values(ascending=False)
        recommendations = {project: round(user_predictions[project], 2) for project in user_predictions.head(num_recommendations).index}
        return recommendations

    # Train-test split for evaluation
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

    # Calculate Mean Squared Error (MSE) for evaluation
    def calculate_mse():
        predictions, actuals = [], []
        for _, row in test_data.iterrows():
            user, project, actual_investment = row['user_id'], row['project_id'], row['investment_amount']
            if project in predicted_df.columns and user in predicted_df.index:
                predicted_investment = predicted_df.loc[user, project]
                predictions.append(predicted_investment)
                actuals.append(actual_investment)
        return mean_squared_error(actuals, predictions)

    # Print Recommendations and MSE
    print(f"Top recommended projects and investment amounts for user {user_id}:")
    recommendations = recommend_projects(user_id, num_recommendations)
    if recommendations:
        for project, investment in recommendations.items():
            print(f"Project ID: {project}, Predicted Investment Amount: {investment}")
    
    # Calculate and print the Mean Squared Error
    mse = calculate_mse()
    print(f"\nMean Squared Error: {mse}")

# Usage: Specify the user ID
train_and_recommend(user_id=1)
